#Tutorial: Introdução aos Serviços S3 e SQS da AWS e Como Usá-los com boto3

Neste tutorial, vamos explorar os serviços Amazon S3 e Amazon SQS da AWS e aprender como interagir com esses serviços usando o SDK do Python, o boto3. Ambos os serviços são amplamente utilizados na arquitetura de aplicações na nuvem e oferecem funcionalidades robustas para armazenamento de arquivos e gerenciamento de filas de mensagens, respectivamente.

**Pré-requisitos:**

*  Conta AWS com permissões adequadas para acessar o S3 e o SQS.
*  Python 3.x instalado.
*  Biblioteca boto3 instalada (pip install boto3).

## Configurando o Cliente AWS com boto3

Antes de começarmos a interagir com os serviços S3 e SQS, precisamos criar um cliente boto3 que autentica utilizando as credenciais da AWS (Access Key ID e Secret Access Key). Aqui está como podemos fazer isso:

In [ ]:
# Instalando a dependência do boto3
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
import boto3

def criar_cliente(servico, access_key, secret_key, region):
    """
    Função para criar um cliente boto3 para qualquer serviço AWS.

    :param servico: Nome do serviço AWS (ex: 's3', 'sqs')
    :param access_key: Sua AWS Access Key ID
    :param secret_key: Sua AWS Secret Access Key
    :param region: Região da AWS (ex: 'us-east-1')

    :return: Cliente boto3 para o serviço especificado
    """
    cliente = boto3.client(
        servico,
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        region_name=region
    )
    return cliente


## Parte 1: Amazon S3
O Amazon S3 (Simple Storage Service) é uma solução de armazenamento de objetos altamente escalável e durável. Ele permite armazenar e recuperar qualquer quantidade de dados a qualquer momento.

**Criando o Cliente S3**

Usando a função criar_cliente, podemos criar um cliente para o S3:

In [ ]:
# Insira suas credenciais aqui
ACCESS_KEY = '---' # Removido para realizar Commit através do GitHub
SECRET_KEY = '---' # Removido para realizar Commit através do GitHub
REGION = 'us-east-1'

s3_cliente = criar_cliente('s3', ACCESS_KEY, SECRET_KEY, REGION)


**1. Criando um Bucket**

Para armazenar dados no S3, você precisa de um "bucket". Um bucket é basicamente um contêiner para seus objetos (arquivos).

In [ ]:
# Mude o nome do bucket todas as vezes que você for executar esse script para um
# nome novo e não repetido
bucket_name = 'andre-victor-uninassau'

# Criar um bucket
s3_cliente.create_bucket(Bucket=bucket_name)
print(f"Bucket '{bucket_name}' criado com sucesso.")

Bucket 'andre-victor-uninassau' criado com sucesso.


**2. Fazendo Upload de um Arquivo**

Você pode fazer upload de arquivos no bucket do S3 facilmente com o método upload_file.

In [ ]:
# Caminho para o arquivo local
arquivo_local = '/content/sample_data/anscombe.json'

# Nome do arquivo no S3
arquivo_no_s3 = 'anscombe.json'

# Fazer o upload
s3_cliente.upload_file(arquivo_local, bucket_name, arquivo_no_s3)
print(f"Arquivo '{arquivo_local}' enviado como '{arquivo_no_s3}' no bucket '{bucket_name}'.")


Arquivo '/content/sample_data/anscombe.json' enviado como 'anscombe.json' no bucket 'andre-victor-uninassau'.


**3. Listando Objetos no Bucket**

Para ver quais arquivos estão em um bucket:

In [ ]:
# Listar objetos no bucket
response = s3_cliente.list_objects_v2(Bucket=bucket_name)

# Verificar se existem objetos
if 'Contents' in response:
    for obj in response['Contents']:
        print(f"Arquivo encontrado: {obj['Key']}")
else:
    print("Nenhum arquivo encontrado no bucket.")


Arquivo encontrado: anscombe.json


**4. Fazendo Download de um Arquivo**

Para baixar um arquivo do S3 para o seu sistema local:

In [ ]:
# Nome do arquivo no S3 e o caminho local onde será salvo
arquivo_para_download = 'anscombe.json'
caminho_local = '/content/sample_data/anscombe-copy.json'

# Fazer o download
s3_cliente.download_file(bucket_name, arquivo_para_download, caminho_local)
print(f"Arquivo '{arquivo_para_download}' baixado para '{caminho_local}'.")


Arquivo 'anscombe.json' baixado para '/content/sample_data/anscombe-copy.json'.


Maiores informações sobre o serviço: https://aws.amazon.com/pt/s3/

#Parte 2: Amazon SQS

O Amazon SQS (Simple Queue Service) é um serviço de mensagens distribuídas que permite que as aplicações se comuniquem de forma desacoplada e assíncrona.

**Criando o Cliente SQS**

Agora, vamos criar um cliente para o serviço SQS usando a mesma função criar_cliente:

In [ ]:
sqs_cliente = criar_cliente('sqs', ACCESS_KEY, SECRET_KEY, REGION)

**1. Criando uma Fila**

Para criar uma fila no SQS, usamos o método create_queue.

In [ ]:
nome_da_fila = 'minha-fila-uninassau'

response = sqs_cliente.create_queue(QueueName=nome_da_fila)
queue_url = response['QueueUrl']
print(f"Fila '{nome_da_fila}' criada com sucesso. URL da fila: {queue_url}")

Fila 'minha-fila-uninassau' criada com sucesso. URL da fila: https://sqs.us-east-1.amazonaws.com/730335451097/minha-fila-uninassau


**2. Enviando uma Mensagem**

Para enviar uma mensagem para a fila:

In [ ]:
mensagem = 'Olá, esta é uma mensagem de exemplo!'

response = sqs_cliente.send_message(QueueUrl=queue_url, MessageBody=mensagem)
print(f"Mensagem enviada com sucesso! ID da mensagem: {response['MessageId']}")

Mensagem enviada com sucesso! ID da mensagem: c93fc9b6-9063-435f-8297-f0b7770bf13d


**3. Recebendo Mensagens**

Podemos recuperar mensagens de uma fila com o método receive_message.

In [ ]:
response = sqs_cliente.receive_message(QueueUrl=queue_url, MaxNumberOfMessages=1)

# Verificar se há mensagens
if 'Messages' in response:
    mensagem_recebida = response['Messages'][0]
    print(f"Mensagem recebida: {mensagem_recebida['Body']}")

    # Excluir a mensagem da fila
    sqs_cliente.delete_message(QueueUrl=queue_url, ReceiptHandle=mensagem_recebida['ReceiptHandle'])
    print("Mensagem excluída da fila.")
else:
    print("Nenhuma mensagem disponível.")

Mensagem recebida: Olá, esta é uma mensagem de exemplo!
Mensagem excluída da fila.


**4. Excluindo uma Fila**

Para excluir a fila quando ela não for mais necessária:

Maiores informações : https://aws.amazon.com/pt/sqs/

# Introdução aos Serviços SNS

Nesta continuação do tutorial, abordaremos o Amazon SNS (Simple Notification Service) . O SNS é usado para enviar notificações em tempo real a serviços ou usuários.

**Pré-requisitos:**

*  Conta AWS com permissões adequadas para acessar o SNS e o Rekognition.
*  Python 3.x instalado.
*  Biblioteca boto3 instalada (pip install boto3).


**Parte 1: Amazon SNS (Simple Notification Service)**

O Amazon SNS é um serviço que permite o envio de mensagens de forma eficiente e escalável para dispositivos móveis, serviços e endpoints HTTP/HTTPS.

**Criando o Cliente SNS**

Usaremos a mesma função criar_cliente para criar o cliente para o SNS:

In [ ]:
sns_cliente = criar_cliente('sns', ACCESS_KEY, SECRET_KEY, REGION)

**1. Criando um Tópico SNS**

Um tópico SNS é o ponto central de distribuição de mensagens para os assinantes (subscribers). Podemos criar um tópico SNS da seguinte forma:

In [ ]:
nome_do_topico = 'meu-topico-exemplo'

response = sns_cliente.create_topic(Name=nome_do_topico)
topic_arn = response['TopicArn']
print(f"Tópico '{nome_do_topico}' criado com sucesso. ARN do tópico: {topic_arn}")

Tópico 'meu-topico-exemplo' criado com sucesso. ARN do tópico: arn:aws:sns:us-east-1:730335451097:meu-topico-exemplo


**2. Inscrevendo um Assinante (Subscriber)**

Para inscrever um assinante (por exemplo, um endereço de email) no tópico criado:

In [ ]:
email_assinante = 'lipe.nscm@gmail.com'

response = sns_cliente.subscribe(
    TopicArn=topic_arn,
    Protocol='email',  # O protocolo pode ser email, sms, sqs, etc.
    Endpoint=email_assinante
)
subscription_arn = response['SubscriptionArn']
print(f"Assinante '{email_assinante}' inscrito com sucesso. ARN da inscrição: {subscription_arn}")


Assinante 'lipe.nscm@gmail.com' inscrito com sucesso. ARN da inscrição: pending confirmation


**3. Enviando uma Mensagem para o Tópico**

Agora podemos enviar uma mensagem para todos os assinantes do tópico:

In [ ]:
mensagem = 'Esta é uma mensagem de exemplo enviada pelo SNS!'

sns_cliente.publish(
    TopicArn=topic_arn,
    Message=mensagem
)
print(f"Mensagem enviada para o tópico '{nome_do_topico}'.")

Mensagem enviada para o tópico 'meu-topico-exemplo'.


**4. Excluindo um Tópico SNS**

Quando o tópico SNS não for mais necessário, podemos excluí-lo da seguinte forma:

In [ ]:
sns_cliente.delete_topic(TopicArn=topic_arn)
print(f"Tópico '{nome_do_topico}' excluído com sucesso.")


Tópico 'meu-topico-exemplo' excluído com sucesso.


Maiores informações : https://aws.amazon.com/pt/sns/

# Introdução aos Serviços Lambda e EC2 da AWS com boto3

Nesta seção, vamos explorar dois serviços essenciais da AWS: AWS Lambda, um serviço de computação sem servidor que executa seu código em resposta a eventos, e Amazon EC2 (Elastic Compute Cloud), que fornece capacidade de computação escalável na nuvem. Veremos como usar o boto3 para interagir com esses serviços.

**Pré-requisitos:**

*  Conta AWS com permissões adequadas para acessar o Lambda e o EC2.
*  Python 3.x instalado.
*  Biblioteca boto3 instalada (pip install boto3).

**Configurando o Cliente AWS com boto3**

Como nos tutoriais anteriores, utilizaremos a função criar_cliente para criar os clientes do AWS Lambda e EC2:

In [ ]:
import boto3

def criar_cliente(servico, access_key, secret_key, region):
    """
    Função para criar um cliente boto3 para qualquer serviço AWS.
    """
    cliente = boto3.client(
        servico,
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        region_name=region
    )
    return cliente


## Parte 1: AWS Lambda

O AWS Lambda permite que você execute código sem precisar provisionar ou gerenciar servidores. Você só precisa enviar o código e o Lambda o executa automaticamente em resposta a eventos.

**Criando o Cliente Lambda**

Para começar, vamos criar um cliente boto3 para o Lambda:

In [ ]:
lambda_cliente = criar_cliente('lambda', ACCESS_KEY, SECRET_KEY, REGION)

**1. Criando uma Função Lambda**

Você pode criar uma função Lambda com o código-fonte e definir suas configurações. Aqui está um exemplo de como criar uma função simples que retorna "Hello World".

In [ ]:
import zipfile

# Função para criar um pacote de código zip para Lambda
def criar_pacote_lambda(arquivo_python, arquivo_zip):
    with zipfile.ZipFile(arquivo_zip, 'w') as zf:
        zf.write(arquivo_python)

# Nome do arquivo Python e pacote zip
arquivo_python = 'lambda_function.py'
arquivo_zip = 'lambda_package.zip'

# Criar o pacote zipado da função
criar_pacote_lambda(arquivo_python, arquivo_zip)

# Configuração da função Lambda
nome_funcao = 'minhaFuncaoLambda'
role_arn = 'arn:aws:iam::123456789012:role/service-role/lambda-role'

# Carregar o código zipado
with open(arquivo_zip, 'rb') as f:
    codigo_zip = f.read()

# Criar a função Lambda
response = lambda_cliente.create_function(
    FunctionName=nome_funcao,
    Runtime='python3.8',
    Role=role_arn,
    Handler='lambda_function.lambda_handler',  # Handler padrão
    Code={'ZipFile': codigo_zip},
    Timeout=15,
    MemorySize=128
)

print(f"Função Lambda '{nome_funcao}' criada com sucesso.")

No arquivo lambda_function.py, você pode ter o seguinte código simples:


In [ ]:
# lambda_function.py
def lambda_handler(event, context):
    return {
        'statusCode': 200,
        'body': 'Hello World!'
    }


**2. Invocando uma Função Lambda**

Você pode invocar uma função Lambda diretamente pelo boto3:

In [ ]:
response = lambda_cliente.invoke(
    FunctionName=nome_funcao,
    InvocationType='RequestResponse',
    Payload=b'{}'  # Payload vazio, apenas um exemplo
)

resultado = response['Payload'].read().decode('utf-8')
print(f"Resultado da invocação: {resultado}")


**3. Atualizando o Código de uma Função Lambda**

Caso você queira atualizar o código de uma função Lambda, pode fazer isso assim:

In [ ]:
with open(arquivo_zip, 'rb') as f:
    codigo_zip = f.read()

# Atualizar o código da função Lambda
lambda_cliente.update_function_code(
    FunctionName=nome_funcao,
    ZipFile=codigo_zip
)

print(f"Código da função '{nome_funcao}' atualizado com sucesso.")


**4. Excluindo uma Função Lambda**

Para excluir uma função Lambda quando ela não for mais necessária:

In [ ]:
lambda_cliente.delete_function(FunctionName=nome_funcao)
print(f"Função Lambda '{nome_funcao}' excluída com sucesso.")

Maiores informações : https://aws.amazon.com/pt/lambda/

##Parte 2: Amazon EC2
O Amazon EC2 (Elastic Compute Cloud) permite que você lance e gerencie servidores virtuais (chamados de instâncias EC2) na nuvem da AWS.

**Criando o Cliente EC2**

Vamos criar um cliente boto3 para o EC2:

In [ ]:
ec2_cliente = criar_cliente('ec2', ACCESS_KEY, SECRET_KEY, REGION)

**1. Iniciando uma Instância EC2**

Aqui está como você pode iniciar uma instância EC2 usando boto3. Você precisará especificar o AMI ID (Amazon Machine Image), que é a imagem da máquina que será usada para inicializar a instância, e o ID da chave SSH para acessar a instância.

In [ ]:
# Parâmetros para a instância EC2
ami_id = 'ami-0fff1b9a61dec8a5f'  # Substitua com uma AMI válida
tipo_instancia = 't1.micro'
key_name = 'meu-par-de-chaves-2'

# Iniciar uma instância EC2
response = ec2_cliente.run_instances(
    ImageId=ami_id,
    InstanceType=tipo_instancia,
    KeyName=key_name,
    MinCount=1,
    MaxCount=1
)

instancia_id = response['Instances'][0]['InstanceId']
print(f"Instância EC2 '{instancia_id}' iniciada com sucesso.")


Instância EC2 'i-05e6d2cd77d392354' iniciada com sucesso.


**2. Listando Instâncias EC2**

Você pode listar todas as suas instâncias EC2 da seguinte forma:

In [ ]:
response = ec2_cliente.describe_instances()

for reservation in response['Reservations']:
    for instance in reservation['Instances']:
        print(f"Instância ID: {instance['InstanceId']} - Estado: {instance['State']['Name']}")


Instância ID: i-05e6d2cd77d392354 - Estado: running


**3. Parando uma Instância EC2**

Para parar uma instância em execução:

In [ ]:
response = ec2_cliente.stop_instances(InstanceIds=[instancia_id])
print(f"Instância EC2 '{instancia_id}' sendo parada.")


Instância EC2 'i-05e6d2cd77d392354' sendo parada.


**4. Iniciando uma Instância EC2**

Caso a instância esteja parada, você pode reiniciá-la:

In [ ]:
response = ec2_cliente.start_instances(InstanceIds=[instancia_id])
print(f"Instância EC2 '{instancia_id}' sendo iniciada.")

**5. Excluindo (Terminando) uma Instância EC2**

Quando você não precisa mais de uma instância, pode encerrá-la para evitar custos adicionais:

In [ ]:
response = ec2_cliente.terminate_instances(InstanceIds=[instancia_id])
print(f"Instância EC2 '{instancia_id}' sendo terminada.")

Instância EC2 'i-05e6d2cd77d392354' sendo terminada.


Maiores informações : https://aws.amazon.com/pt/ec2/